In [1]:
import requests
session = requests.session()
# base_url = "http://localhost:8000"
base_url = "https://maziar-temp-project.ir/connect4"

In [2]:
#TODO Change the emails to your own email and your opponent emain and please go to https://maziarheidari2003.github.io/connect4game and make an account first

# This is the bot player
my_auth_data = session.post(f"{base_url}/api/v1/auth/login", json={"email":"maziarheidari1124@gmail.com"}).json()
my_id = my_auth_data["id"]
my_token ="Bearer " + my_auth_data["access_token"]

# This is the player who should go to the browser after creating and joining the game
opponent_auth_data = session.post(f"{base_url}/api/v1/auth/login", json={"email":"babak@gmail.com"}).json()
opponent_id = opponent_auth_data["id"]
opponent_token = "Bearer " + opponent_auth_data["access_token"]


In [8]:
created_game_uuid = session.post(f"{base_url}/api/v1/game/",  headers={"Authorization": my_token}).json()

In [4]:
import random

def my_make_move(token, game_uuid, opponent_last_move: int | None = None):
    if opponent_last_move is None: 
        # In case you are the first player
        pass
    else:
        #Otherwise you are going to make a move based on the opponents last move
        pass
    col = random.randint(0, 6)
    res = session.patch(
        f"{base_url}/api/v1/game/make-move",
        params={"game_uuid": game_uuid, "chosen_column": col},
        headers={"Authorization": token}
    )

In [9]:
import json, websockets, asyncio

async def human_vs_bot():
    uri = f"wss://maziar-temp-project.ir/connect4/ws/api/v1/game/{created_game_uuid}"

    async with websockets.connect(uri) as ws:
        print("WebSocket connected")
        game = session.patch(f"{base_url}/api/v1/game/join?game_uuid={created_game_uuid}", headers={"Authorization": opponent_token}).json()
        # If I place the join request outside of this context, the socket would never publish data for the first time
       
        print("You have 20 second to join the game dear human being! Visit https://maziarheidari2003.github.io/connect4game ")

        msg = json.loads(await ws.recv())
        print(msg)
        await asyncio.sleep(20) 

        while True:
            status = msg["status"]
            if status == "FINISHED":
                print("Game finished. Winner:", msg["winner"])
                break

            current_turn = msg.get("current_turn_id")
            if current_turn == my_id: 
                my_make_move(my_token, created_game_uuid, msg.get("last_move"))
            
            msg = json.loads(await ws.recv())
    

await human_vs_bot()


WebSocket connected
You have 20 second to join the game dear human being! Visit https://maziarheidari2003.github.io/connect4game 
{'board': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'status': 'IN_PROGESS', 'current_turn': 'maziar', 'current_turn_id': 1, 'winner': None, 'moves_count': 0}
Game finished. Winner: babak
